# Miscellanea

In [1]:
import ray
ray.shutdown()

In [2]:
import os
from pathlib import Path

# This is a terrible hack, and very unsafe
while not Path("config_data").exists():
    os.chdir("..")

In [3]:
print(os.getcwd())

/home/enrico/slim


In [4]:
# If you got the artifacts please put them inside outputs
from pathlib import Path
from slim.simulation.config import Config
from slim.simulation.simulator import Simulator, _get_simulation_path, load_artifact

def load_data(config_folder: str, sim_name: str, farms_per_process=1, **kwargs):
    cfg_data_path = Path("config_data")
    cfg_data = cfg_data_path / "config.json"
    config_folder = cfg_data_path / config_folder
    outputs_path = Path("outputs")
    
    artifact = _get_simulation_path(outputs_path, sim_name)[0]
    if not artifact.exists():
        print(f"Artifact not yet generated, running {sim_name} with {kwargs} options")
        # defection_proba is farm-specific and needs special handling
        defection_proba = kwargs.pop("defection_proba", None)
        cfg = Config(cfg_data, config_folder, farms_per_process=farms_per_process, **kwargs)

        if defection_proba is not None:
            for farm in cfg.farms:
                farm.defection_proba = defection_proba
        
        sim = Simulator(outputs_path, sim_name, cfg)
        sim.run_model()
    
    return load_artifact(outputs_path, sim_name)

    #print(config_folder, outputs)

In [5]:
bernoullis = [load_data("Fyne_complete", f"bernoulli_{i/10}", treatment_strategy="bernoulli", defection_proba=i / 10)  for i in range(11)]

/home/enrico/slim/slim/simulation/lice_population.py:629: RuntimeWarning: invalid value encountered in true_divide
  probas = config_matrix / sums


Artifact not yet generated, running bernoulli_0.9 with {'treatment_strategy': 'bernoulli', 'defection_proba': 0.9} options


100%|██████████████████████████████████████████████████████████████████████████████████████| 730/730 [01:51<00:00,  6.54it/s]


Stopping farms
Artifact not yet generated, running bernoulli_1.0 with {'treatment_strategy': 'bernoulli', 'defection_proba': 1.0} options


100%|██████████████████████████████████████████████████████████████████████████████████████| 730/730 [01:47<00:00,  6.78it/s]


Stopping farms


In [6]:
mosaic = load_data("Fyne_complete", "mosaic", treatment_strategy="mosaic")

Artifact not yet generated, running mosaic with {'treatment_strategy': 'mosaic'} options


100%|██████████████████████████████████████████████████████████████████████████████████████| 730/730 [01:38<00:00,  7.41it/s]


Stopping farms


In [8]:
# for example
mosaic[0]

aggregation  \
timestamp  farm_id                                                      
2018-01-01 farm_0   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
           farm_1   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
           farm_2   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
           farm_3   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
           farm_4   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
...                                                               ...   
2019-12-31 farm_4   [0.00819836, 0.0088176355, 0.009518084, 0.0098...   
           farm_5   [0.010761421, 0.010773453, 0.010357433, 0.0101...   
           farm_6   [0.03230719, 0.03329344, 0.03208614, 0.0335183...   
           farm_7   [0.00807184, 0.007969333, 0.008980827, 0.00746...   
           farm_8   [0.0017031423, 0.001618123, 0.0017867779, 0.00...   

                   reported_aggregation  \
timestamp  farm_id                        
2018-01-01 farm_0                 [0.0]   
           farm_1                 [0.0]   
           farm_2                 [0.0]   
           farm_3                 [0.0]   
           farm_4                 [0.0]   
...                                 ...   
2019-12-31 farm_4                 [0.0]   
           farm_5                 [0.0]   
           farm_6                 [0.0]   
           farm_7                 [0.0]   
           farm_8                 [0.0]   

                                                      fish_population  \
timestamp  farm_id                                                      
2018-01-01 farm_0   [40000, 40000, 40000, 40000, 40000, 40000, 400...   
           farm_1   [40000, 40000, 40000, 40000, 40000, 40000, 400...   
           farm_2   [40000, 40000, 40000, 40000, 40000, 40000, 400...   
           farm_3   [40000, 40000, 40000, 40000, 40000, 40000, 400...   
           farm_4   [40000, 40000, 40000, 40000, 40000, 40000, 400...   
...                                                               ...   
2019-12-31 farm_4   [10002, 9980, 9981, 9986, 9988, 9987, 9988, 99...   
           farm_5   [9850, 9839, 9848, 9856, 9838, 9844, 9847, 985...   
           farm_6   [9193, 9191, 9194, 9189, 9188, 9190, 9191, 919...   
           farm_7   [9911, 9913, 9910, 9909, 9908, 9911, 9903, 991...   
           farm_8   [11743, 11742, 11753, 11752, 11739, 11740, 117...   

                   current_treatments  allowed_treatments asked_to_treat  \
timestamp  farm_id                                                         
2018-01-01 farm_0           [0, 0, 0]                  10            [0]   
           farm_1           [0, 0, 0]                   8            [0]   
           farm_2           [0, 0, 0]                  10            [0]   
           farm_3           [0, 0, 0]                  10            [0]   
           farm_4           [0, 0, 0]                  10            [0]   
...                               ...                 ...            ...   
2019-12-31 farm_4           [0, 0, 0]                   1            [0]   
           farm_5           [0, 0, 0]                   1            [0]   
           farm_6           [0, 0, 0]                   1            [0]   
           farm_7           [0, 0, 0]                   1            [0]   
           farm_8           [0, 0, 0]                   1            [0]   

                    new_reservoir_lice  \
timestamp  farm_id                       
2018-01-01 farm_0                  150   
           farm_1                  150   
           farm_2                  150   
           farm_3                  150   
           farm_4                  150   
...                                ...   
2019-12-31 farm_4                 2153   
           farm_5                 2153   
           farm_6                 2153   
           farm_7                 2153   
           farm_8                 2153   

                                            new_reservoir_lice

In [11]:
mosaic_df = mosaic[0]
# aggregation: list of aggregations per cage (i.e. agg_i = AF_i/N_i for every cage i)
# reported_aggregation: a sampled aggregation. Will be different from 1 only when reporting
# fish_population: fish population per cage
# current_treatments: bitset denoting which treatment is being used (0th->EMB, 1st->Thermolicer, 2nd->Fallowing)
mosaic_df

aggregation  \
timestamp  farm_id                                                      
2018-01-01 farm_0   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
           farm_1   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
           farm_2   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
           farm_3   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
           farm_4   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
...                                                               ...   
2019-12-31 farm_4   [0.00819836, 0.0088176355, 0.009518084, 0.0098...   
           farm_5   [0.010761421, 0.010773453, 0.010357433, 0.0101...   
           farm_6   [0.03230719, 0.03329344, 0.03208614, 0.0335183...   
           farm_7   [0.00807184, 0.007969333, 0.008980827, 0.00746...   
           farm_8   [0.0017031423, 0.001618123, 0.0017867779, 0.00...   

                   reported_aggregation  \
timestamp  farm_id                        
2018-01-01 farm_0                 [0.0]   
           farm_1                 [0.0]   
           farm_2                 [0.0]   
           farm_3                 [0.0]   
           farm_4                 [0.0]   
...                                 ...   
2019-12-31 farm_4                 [0.0]   
           farm_5                 [0.0]   
           farm_6                 [0.0]   
           farm_7                 [0.0]   
           farm_8                 [0.0]   

                                                      fish_population  \
timestamp  farm_id                                                      
2018-01-01 farm_0   [40000, 40000, 40000, 40000, 40000, 40000, 400...   
           farm_1   [40000, 40000, 40000, 40000, 40000, 40000, 400...   
           farm_2   [40000, 40000, 40000, 40000, 40000, 40000, 400...   
           farm_3   [40000, 40000, 40000, 40000, 40000, 40000, 400...   
           farm_4   [40000, 40000, 40000, 40000, 40000, 40000, 400...   
...                                                               ...   
2019-12-31 farm_4   [10002, 9980, 9981, 9986, 9988, 9987, 9988, 99...   
           farm_5   [9850, 9839, 9848, 9856, 9838, 9844, 9847, 985...   
           farm_6   [9193, 9191, 9194, 9189, 9188, 9190, 9191, 919...   
           farm_7   [9911, 9913, 9910, 9909, 9908, 9911, 9903, 991...   
           farm_8   [11743, 11742, 11753, 11752, 11739, 11740, 117...   

                   current_treatments  allowed_treatments asked_to_treat  \
timestamp  farm_id                                                         
2018-01-01 farm_0           [0, 0, 0]                  10            [0]   
           farm_1           [0, 0, 0]                   8            [0]   
           farm_2           [0, 0, 0]                  10            [0]   
           farm_3           [0, 0, 0]                  10            [0]   
           farm_4           [0, 0, 0]                  10            [0]   
...                               ...                 ...            ...   
2019-12-31 farm_4           [0, 0, 0]                   1            [0]   
           farm_5           [0, 0, 0]                   1            [0]   
           farm_6           [0, 0, 0]                   1            [0]   
           farm_7           [0, 0, 0]                   1            [0]   
           farm_8           [0, 0, 0]                   1            [0]   

                    new_reservoir_lice  \
timestamp  farm_id                       
2018-01-01 farm_0                  150   
           farm_1                  150   
           farm_2                  150   
           farm_3                  150   
           farm_4                  150   
...                                ...   
2019-12-31 farm_4                 2153   
           farm_5                 2153   
           farm_6                 2153   
           farm_7                 2153   
           farm_8                 2153   

                                            new_reservoir_lice